# Pre-processing data and create model for Saina

## Importing packages

In [1]:
import os
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Reading dataset

In [2]:
saina = pd.read_csv(f"{'/'.join(os.getcwd().split('/')[:-2])}/datasets/saina.csv")

In [3]:
saina

,0,1,2,3,4,5,6
0,ساینا،,1401,دنده ای-S,کارکرد صفر,دنده ای,سفید صدفی,"325,000,000"
1,ساینا،,1400,دنده ای-S,"14,000 کیلومتر",دنده ای,سفید,"300,000,000"
2,ساینا،,1399,دنده ای-EX,"46,000 کیلومتر",دنده ای,سفید,"290,000,000"
3,ساینا،,1401,دنده ای-S,کارکرد صفر,دنده ای,سفید,"320,000,000"
4,ساینا،,1399,دنده ای-EX,"14,000 کیلومتر",دنده ای,نوک مدادی,"295,000,000"
...,...,...,...,...,...,...,...
147,ساینا،,1401,دنده ای-EX,"6,200 کیلومتر",دنده ای,سفید,"297,000,000"
148,ساینا،,1398,دنده ای-EX,"18,000 کیلومتر",دنده ای,سفید,"270,000,000"
149,ساینا،,1399,دنده ای-EX,"47,000 کیلومتر",دنده ای,سفید,"275,000,000"
150,ساینا،,1401,دنده ای-S,کارکرد صفر,دنده ای,سفید,"304,000,000"


## Change columns name

In [4]:
def edit_cols_name(df):
    df.rename(columns={'0': 'نام خودرو'}, inplace=True)
    df.rename(columns={'1': 'سال تولید'}, inplace=True)
    df.rename(columns={'2': 'مدل'}, inplace=True)
    df.rename(columns={'3': 'کارکرد'}, inplace=True)
    df.rename(columns={'4': 'گیربکس'}, inplace=True)
    df.rename(columns={'5': 'رنگ'}, inplace=True)
    df.rename(columns={'6': 'قیمت'}, inplace=True)
    
edit_cols_name(saina)

## Convert km to standard mode

In [5]:
def fix_km(df):
    for index in range(len(df['کارکرد'])):
        if df['کارکرد'][index].strip() == 'کارکرد صفر':
            df['کارکرد'][index] = 0
        elif 'کیلومتر' in df['کارکرد'][index]:
            df['کارکرد'][index] = int(df['کارکرد'][index].split()[0].replace(',', ''))
                   
fix_km(saina)

## Convert price to standard mode

In [6]:
for i in range(len(saina)):
    saina['قیمت'][i] = saina['قیمت'][i].replace(',', '')
    saina['قیمت'][i] = int(saina['قیمت'][i])

## Check different car models and organize them

In [7]:
saina['مدل'].value_counts()

دنده ای-EX           79
دنده ای-S            66
دنده ای-پلاس          5
اتوماتیک-اتوماتیک     2
Name: مدل, dtype: int64

In [8]:
for i in range(len(saina)):
    saina['مدل'][i] = saina['مدل'][i].split('-')[1]
    
saina.drop(saina[saina['مدل'] == 'پلاس'].index, axis=0, inplace=True)
saina.drop(saina[saina['مدل'] == 'اتوماتیک'].index, axis=0, inplace=True)

In [9]:
saina['مدل'].value_counts()

EX    79
S     66
Name: مدل, dtype: int64

## Delete the especially missing value

In [10]:
saina.drop(saina[saina['کارکرد'] == 'کارکرده'].index, axis=0, inplace=True)

## Check different car colors and organize them

In [11]:
saina['رنگ'].value_counts()

سفید         140
نوک مدادی      2
مشکی           1
سفید صدفی      1
آبی            1
Name: رنگ, dtype: int64

In [12]:
saina.drop(saina[saina['رنگ'] == 'نوک مدادی'].index, axis=0, inplace=True)
saina.drop(saina[saina['رنگ'] == 'مشکی'].index, axis=0, inplace=True)
saina.drop(saina[saina['رنگ'] == 'آبی'].index, axis=0, inplace=True)
saina.drop(saina[saina['رنگ'] == 'سفید صدفی'].index, axis=0, inplace=True)

In [13]:
saina['رنگ'].value_counts()

سفید    140
Name: رنگ, dtype: int64

## Remove extra fields

In [14]:
saina.drop(['نام خودرو'], axis=1, inplace=True)

In [15]:
saina

,سال تولید,مدل,کارکرد,گیربکس,رنگ,قیمت
1,1400,S,14000,دنده ای,سفید,300000000
2,1399,EX,46000,دنده ای,سفید,290000000
3,1401,S,0,دنده ای,سفید,320000000
5,1396,EX,133000,دنده ای,سفید,257000000
6,1401,S,0,دنده ای,سفید,307000000
...,...,...,...,...,...,...
147,1401,EX,6200,دنده ای,سفید,297000000
148,1398,EX,18000,دنده ای,سفید,270000000
149,1399,EX,47000,دنده ای,سفید,275000000
150,1401,S,0,دنده ای,سفید,304000000


## One-hot fields

In [16]:
def make_one_hot(data, field):
    temp = pd.get_dummies(data[field], prefix=field)
    data.drop(field, axis=1, inplace=True)
    return pd.concat([data, temp], axis=1)

In [17]:
saina = make_one_hot(saina, 'مدل')
saina = make_one_hot(saina, 'گیربکس')
saina = make_one_hot(saina, 'رنگ')

In [18]:
saina

,سال تولید,کارکرد,قیمت,مدل_EX,مدل_S,گیربکس_دنده ای,رنگ_سفید
1,1400,14000,300000000,0,1,1,1
2,1399,46000,290000000,1,0,1,1
3,1401,0,320000000,0,1,1,1
5,1396,133000,257000000,1,0,1,1
6,1401,0,307000000,0,1,1,1
...,...,...,...,...,...,...,...
147,1401,6200,297000000,1,0,1,1
148,1398,18000,270000000,1,0,1,1
149,1399,47000,275000000,1,0,1,1
150,1401,0,304000000,0,1,1,1


## Remove the label

In [19]:
labels = saina['قیمت']
saina.drop(['قیمت'], axis=1, inplace=True)

## Separating data into train and test sections

In [20]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(saina, labels, test_size=0.1, shuffle=True)

## Model making

In [21]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

saina_model = LinearRegression()
saina_model.fit(x_train, y_train)
prediction = saina_model.predict(x_test)

error = int(mean_absolute_error(y_test, prediction))
print(f'{error:,} MT')

6,166,519 MT


## Save the model

In [22]:
import pickle
pickle.dump(saina_model, open('saina_model.sav', 'wb'))